In [7]:
import nest_asyncio
import twint
import pandas as pd
from green_mood_tracker.data_cleaning import clean

nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
sys.path.insert(0, '../green_mood_tracker/')
from twint_class import TWINT

In [9]:
kwargs = dict(
    keywords=['fossil fuels'],
    cities=['London'],
    since = '2020-11-11 12:00:00',
    limit=100
)

t = TWINT(**kwargs)

df_city = t.city_df()

In [10]:
df_city_clean = clean(df_city,'tweet')
df_city.head()

,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,language,...,trans_src,translate,tweet,urls,user_id,user_id_str,user_rt,user_rt_id,username,video
0,[],1332343590601764865,1.606490e+12,2020-11-27 15:20:40,5,,[],15,1332343590601764865,en,...,,,biggest emission scandal youve never heard des...,[https://bit.ly/36e1DTW],234933359,234933359,,,ESG_Comms,0
1,[],1332286547467517952,1.606489e+12,2020-11-27 14:56:19,5,,[],14,1332337462421594112,en,...,,,expect kind report come oil industry lobbyist ...,[],2983911,2983911,,,captaink99,0
2,[],1332337271496957953,1.606489e+12,2020-11-27 14:55:34,5,,[],14,1332337272973369345,en,...,,,sadly western capitalist model ha trouble thin...,[],797215995014352901,797215995014352901,,,BiodiversitySoS,0
3,[],1332330341613379585,1.606487e+12,2020-11-27 14:28:01,5,,[],14,1332330341613379585,en,...,,,u looking bar large bank declining business pa...,[https://buff.ly/2V2KlTK],2620095853,2620095853,,,DeSmogUK,0
4,[],1332329301887381504,1.606487e+12,2020-11-27 14:23:53,5,,[],14,1332329301887381504,en,...,,,im certainly expert maybe naive acceptable ene...,[],1184029015318941702,1184029015318941702,,,JamesCox1994,1


In [12]:
from tensorflow import keras
model = keras.models.load_model('../models/model_roBERTa_binary_2.tf')

In [14]:
from green_mood_tracker.encoders import RobertaEncoder

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/henryhall/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
ds_twint_encoded = RobertaEncoder(
            df_city.tweet,df_city.day).sentence_encode(32, shuffle=True)

In [33]:
ds_twint_encoded

<BatchDataset shapes: ({input_ids: (None, 30), attention_mask: (None, 30)}, (None, 1)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>

In [23]:
model.summary()

Model: "tf_roberta_for_sequence_classification_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (Custom>TFRobertaMai multiple                  124645632 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592130    
Total params: 125,237,762
Trainable params: 125,237,762
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.predict(ds_train_encoded)

ValueError: in user code:

    /Users/henryhall/.pyenv/versions/3.7.7/envs/lewagon-3.7/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /Users/henryhall/.pyenv/versions/3.7.7/envs/lewagon-3.7/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/henryhall/.pyenv/versions/3.7.7/envs/lewagon-3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/henryhall/.pyenv/versions/3.7.7/envs/lewagon-3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/henryhall/.pyenv/versions/3.7.7/envs/lewagon-3.7/lib/python3.7/site-packages/tensorflo